## Auto Insurance Claims Fraud - Supervised Learning

Nearly one in 10 Americans would commit insurance fraud if they knew they could get away with it. Nearly one in four Americans say it’s ok to defraud insurers.   About one in 10 people agree it’s ok to submit claims for items that aren’t lost or damaged, or for personal injuries that didn’t occur. Two in five people are “not very likely” or “not likely at all” to report someone who ripped of an insurer. 
\- Accenture Ltd.(2003)   

Nearly three of 10 Americans (29 percent) wouldn't report insurance scams committed by someone they know. 
\- Progressive Insurance (2001)

This notebook shows how to "flag" fraudulent insurance claims based on past known cases using several supervised learning algorithms (Random Forest, Decision Tree, SVM, GLM). These models can be evaluated using a variety of metrics, including generating a lift chart. Computations on data occur inside Autonomous Database, leveraging it as a high-performance compute engine.

Copyright (c) 2021 Oracle Corporation 
###### <a href="https://oss.oracle.com/licenses/upl/" onclick="return ! window.open('https://oss.oracle.com/licenses/upl/');">The Universal Permissive License (UPL), Version 1.0</a>
---

 
![tiny arrow](http://www.oracle.com/technetwork/database/options/advanced-analytics/autoinsurancepic60-5434493.jpg "tiny arrow")


## Business Problem

---
Based on "labeled data" derived from automobile insurance claims evaluators (see related Auto Insurance Claims Fraud - Unsupervised Learning OML Notebook), we can now build supervised learning models that better target known insurance claims fraud.   Based on this historical "labeled data" (Fraudfound Yes/No), we would like to build predictive models to detect likely fraudulent auto claims.

In [3]:
%python

import pandas as pd
import oml

In [4]:
%python

# URL of the location of the data in CSV format
url="https://raw.githubusercontent.com/oracle/oracle-db-examples/master/machine-learning/datasets/CLAIMS.csv"

# Create a local Pandas Dataframe
claims_pd = pd.read_csv(url)

# Check the number of rows and columns of the PD
claims_pd.shape

In [5]:
%python

# Ensure a table with that name does not exist
try:
    oml.drop(table='CLAIMS')
except:
    pass
    
claims_pd = claims_pd.rename(columns = {"DAYS:POLICY-ACCIDENT": 'DAYSPOLICYACCIDENT', 'DAYS:POLICY-CLAIM': 'DAYSPOLICYCLAIM', 'ADDRESSCHANGE-CLAIM': 'ADDRESSCHANGECLAIM'})

# Create the table CLAIMS and get back a proxy object CLAIMS_DF    
CLAIMS_DF = oml.create(claims_pd, table = 'CLAIMS')

### Display the newly imported data using both Python and SQL
---


In [7]:
%python

z.show(CLAIMS_DF)

In [8]:
%sql 

SELECT * FROM CLAIMS;

## Data Understanding

---
In this section, we use the built-in notebook visualization capabilities to explore the data.

In [10]:
%sql
SELECT * from CLAIMS;

In [11]:
%sql

SELECT * from CLAIMS;

In [12]:
%sql

SELECT * from CLAIMS;

## Data Preparation

---

In [14]:
%script

-- Split the data into random samples of 60% for training data (CLAIMS_TRAIN_DATA) for training the models 
-- and a 40% hold out sample for testing and evaluating the models (CLAIMS_TEST_DATA)

CREATE OR REPLACE VIEW CLAIMS_TRAIN_DATA AS SELECT * FROM CLAIMS SAMPLE (60) SEED (1);
CREATE OR REPLACE VIEW CLAIMS_TEST_DATA AS SELECT * FROM CLAIMS MINUS SELECT * FROM CLAIMS_TRAIN_DATA;

## Data Transformation using Python 
---

The OML4Py Transparency Layer transforms data in the database instance, without extracting data. 

### One-hot encoding

---

The OML in-database algorithms provide algorithm-specific automatic data preparation. One of the transformations involve categorical columns that must be one-hot encoded (or "exploded"). The following paragraphs demonstrate how to achieve one-hot encoding explicitly using OML4Py on a database table, however, one-hot encoding occurs automatically if PREP_AUTO is set to ON for model building.  


In [17]:
%python

CLAIMS_TRAIN_DF = oml.sync(view = 'CLAIMS_TRAIN_DATA')
categorical_cols = ['MARITALSTATUS', 'MAKE', 'SEX', 'BASEPOLICY']
CLAIMS_TRAIN_CAT_DF = CLAIMS_TRAIN_DF[['POLICYNUMBER'] + categorical_cols]

z.show(CLAIMS_TRAIN_CAT_DF.head(20))

In [18]:
%python

def encode(DF, ids, col):
    assert isinstance(ids, list)
    CNT_DF = DF.crosstab(ids, col)
    PIVOT_DF = CNT_DF.pivot_table(ids, col, 'count', aggfunc = oml.DataFrame.count)
    cols = PIVOT_DF.columns[len(ids):]
    new_columns = [ col + '_' + c.split('_')[1][1:-1] for c in cols]
    new_columns = ids + new_columns
    PIVOT_DF.rename(columns = new_columns)
    return PIVOT_DF

In [19]:
%python

ONEHOT_DF = encode(CLAIMS_TRAIN_CAT_DF, ['POLICYNUMBER'], 'MARITALSTATUS')    

z.show(ONEHOT_DF.head())

In [20]:
%python

RES_DF = CLAIMS_TRAIN_CAT_DF[['POLICYNUMBER']]
for i, col in enumerate(categorical_cols):
    if col == 'POLICYNUMBER':
        continue
    ONEHOT_DF = encode(CLAIMS_TRAIN_CAT_DF, ['POLICYNUMBER'], col)
    RES_DF = RES_DF.merge(ONEHOT_DF, on ='POLICYNUMBER', how = 'inner', suffixes = ['', ''])
    
z.show(RES_DF.head())

In [21]:
%python

cols = RES_DF.columns[1:]
for col in cols:
    RES_DF = RES_DF.replace(old = [None], new = [0.0], default = 1.0, columns = [col])
    
z.show(RES_DF.head())

## Model Building

---
First, we use the Attribute Importance algorithm to rank the attributes (predictor columns) according to how they predict the target, FRAUDFOUND. After filtering columns based on the most predictive, we build 4 models using OML4SQL using RandomForest, GLM, SVM, and Decision Tree in-database algorithms. The default options are used here, however, there are various settings users may provide. Below, we highlight those settings for Decision Tree. 

In [23]:
%script

--  Find the importance of attributes that impact the target FRAUDFOUND

BEGIN DBMS_DATA_MINING.DROP_MODEL('AI_EXPLAIN_OUTPUT_CLAIMS_AI');
EXCEPTION WHEN OTHERS THEN NULL; END;

DECLARE
    V_SETLST DBMS_DATA_MINING.SETTING_LIST;
BEGIN
    V_SETLST('ALGO_NAME')        := 'ALGO_AI_MDL';
    V_SETLST('PREP_AUTO')        := 'ON';

    DBMS_DATA_MINING.CREATE_MODEL2(
        MODEL_NAME        => 'AI_EXPLAIN_OUTPUT_CLAIMS_AI',
        MINING_FUNCTION   => 'ATTRIBUTE_IMPORTANCE',
        DATA_QUERY        => 'SELECT * FROM CLAIMS',
        SET_LIST          => V_SETLST,
        CASE_ID_COLUMN_NAME => 'POLICYNUMBER',
        TARGET_COLUMN_NAME  => 'FRAUDFOUND');
END;


In [24]:
%sql

SELECT * FROM DM$VAAI_EXPLAIN_OUTPUT_CLAIMS_AI ORDER BY ATTRIBUTE_IMPORTANCE_VALUE DESC FETCH FIRST 8 ROWS ONLY;


In [25]:
%sql

-- Add Filtered dataset for Model generation, 
-- keeping Policy Number as well as FraudFound (the Target)

CREATE OR REPLACE VIEW CLAIMS_TRAIN_DATA_FILTERED AS 
SELECT POLICYNUMBER,
       BASEPOLICY, 
       VEHICLECATEGORY, 
       FAULT, 
       PASTNUMBEROFCLAIMS, 
       VEHICLEPRICE, 
       WITNESSPRESENT,
       AGEOFPOLICYHOLDER,
       MARITALSTATUS,
       FRAUDFOUND
FROM CLAIMS_TRAIN_DATA;

In [26]:
%script


BEGIN
    DBMS_DATA_MINING.DROP_MODEL('CLAIMS_CLASS_MODEL_RF');
    EXCEPTION WHEN OTHERS THEN NULL; 
END;

DECLARE
    V_SETLST DBMS_DATA_MINING.SETTING_LIST;
BEGIN
    V_SETLST('PREP_AUTO') := 'ON';
    V_SETLST('ALGO_NAME') := 'ALGO_RANDOM_FOREST';
    DBMS_DATA_MINING.CREATE_MODEL2(
        'CLAIMS_CLASS_MODEL_RF',
        'CLASSIFICATION',
        'SELECT * FROM CLAIMS_TRAIN_DATA_FILTERED',
        V_SETLST,
        'POLICYNUMBER',
        'FRAUDFOUND');
END;



In [27]:
%script


BEGIN
    DBMS_DATA_MINING.DROP_MODEL('CLAIMS_CLASS_MODEL_GLM');
    EXCEPTION WHEN OTHERS THEN NULL; 
END;

DECLARE
    V_SETLST DBMS_DATA_MINING.SETTING_LIST;
BEGIN
    V_SETLST('PREP_AUTO') := 'ON';
    V_SETLST('ALGO_NAME') := 'ALGO_GENERALIZED_LINEAR_MODEL';
    DBMS_DATA_MINING.CREATE_MODEL2(
        'CLAIMS_CLASS_MODEL_GLM',
        'CLASSIFICATION',
        'SELECT * FROM CLAIMS_TRAIN_DATA_FILTERED',
        V_SETLST,
        'POLICYNUMBER',
        'FRAUDFOUND');
END;



In [28]:
%script

BEGIN
    DBMS_DATA_MINING.DROP_MODEL('CLAIMS_CLASS_MODEL_SVM');
    EXCEPTION WHEN OTHERS THEN NULL; 
END;

DECLARE
    V_SETLST DBMS_DATA_MINING.SETTING_LIST;
BEGIN
    V_SETLST('PREP_AUTO') := 'ON';
    V_SETLST('ALGO_NAME') := 'ALGO_SUPPORT_VECTOR_MACHINES';
    DBMS_DATA_MINING.CREATE_MODEL2(
        'CLAIMS_CLASS_MODEL_SVM',
        'CLASSIFICATION',
        'SELECT * FROM CLAIMS_TRAIN_DATA_FILTERED',
        V_SETLST,
        'POLICYNUMBER',
        'FRAUDFOUND');
END;



In [29]:
%script


BEGIN
    DBMS_DATA_MINING.DROP_MODEL('CLAIMS_CLASS_MODEL_DT');
    EXCEPTION WHEN OTHERS THEN NULL; 
END;

DECLARE
    V_SETLST DBMS_DATA_MINING.SETTING_LIST;
BEGIN
    V_SETLST('PREP_AUTO') := 'ON';
    V_SETLST('ALGO_NAME') := 'ALGO_DECISION_TREE';
    DBMS_DATA_MINING.CREATE_MODEL2(
        'CLAIMS_CLASS_MODEL_DT',
        'CLASSIFICATION',
        'SELECT * FROM CLAIMS_TRAIN_DATA_FILTERED',
        V_SETLST,
        'POLICYNUMBER',
        'FRAUDFOUND');
END;



### Additional Options 

---
Above, we have produced a basic model using default hyperparameters. OML in-database algorithms provide a wide range of model hyperparameters, which can be selected and tuned depending on your data and use case.

Please refer to <a href="https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-8987EE6F-41A9-4DF9-997C-129B41FDC59" onclick="return ! window.open('https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-8987EE6F-41A9-4DF9-997C-129B41FDC59');"> this link </a> for more information.

### Examples of possible setting overrides for Decision Tree 

---
*Experiment with tweaking settings for other algorithms as well.*

If the user does not override the default settings, relevant settings are determined by the algorithm.

A complete list of settings can be found in the documentation:

-- Algorithm Settings: <a href="https://oracle.com/goto/ml-decision-tree" onclick="return ! window.open('https://oracle.com/goto/ml-decision-tree');">Decision Tree</a> 

-- Shared Settings: <a href="https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/shared-settings.html#GUID-2EFF3880-F2E2-4449-A8CC-2CF516DD096B" onclick="return ! window.open('https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/shared-settings.html#GUID-2EFF3880-F2E2-4449-A8CC-2CF516DD096B');">All algorithms</a>


-- Specify a row weight column 
    'ODMS_ROW_WEIGHT_COLUMN_NAME' : '<row_weight_column_name>'
   
-- Specify a missing value treatment method for the training data. This setting does not affect the scoring data. The default value is `ODMS_MISSING_VALUE_AUTO`.  The option `ODMS_MISSING_VALUE_MEAN_MODE` replaces missing values with the mean (numeric attributes) or the mode (categorical attributes) both at build time and apply time where appropriate. The option `ODMS_MISSING_VALUE_AUTO` performs different strategies for different algorithms.  When `ODMS_MISSING_VALUE_TREATMENT` is set to `ODMS_MISSING_VALUE_DELETE_ROW`, the rows in the training data that contain missing values are deleted. However, if you want to replicate this missing value treatment in the scoring data, then you must perform the transformation explicitly.
    'ODMS_MISSING_VALUE_TREATMENT' : 'ODMS_MISSING_VALUE_AUTO'
    
-- Specify Tree impurity metric for Decision Tree. 
   Tree algorithms seek the best test question for splitting data at each node. The best splitter and split values are those that result in the largest increase in target value homogeneity (purity) for the entities in the node. Purity is by a metric. Decision trees can use either Gini `TREE_IMPURITY_GINI` or entropy `TREE_IMPURITY_ENTROPY` as the purity metric. By default, the algorithm uses `TREE_IMPURITY_GINI`.
    'TREE_IMPURITY_METRIC' : 'TREE_IMPURITY_GINI'
    
-- Specify the criteria for splits regarding the maximum tree depth (the maximum number of nodes between the root and any leaf node, including the leaf node).
   For Decision Tree, it requires a number between 2 and 20, and the default is 7. For Random Forest it is a number between 2 and 100, and the default is 16.
    'TREE_TERM_MAX_DEPTH' : '7'
    
-- Specify the minimum number of training rows in a node expressed as a percentage of the rows in the training data.
   It requires a number between 0 and 10.  The default is 0.05, indicating 0.05%. 
    'TREE_TERM_MINPCT_NODE' : '0.05'
    
-- Specifyt he minimum number of rows required to consider splitting a node expressed as a percentage of the training rows.
   It requires a number greater than 0, and smaller or equal to 20.  The default is 0.1, indicating 0.1%. 
    'TREE_TERM_MINPCT_SPLIT' : '0.1'

-- Specify The minimum number of rows in a node.
   It requires a number greater than or equal to zero. The default is 10.      
    'TREE_TERM_MINREC_NODE' : '10'
    
-- Specify the criteria for splits regarding the minimum number of records in a parent node expressed as a value. 
   No split is attempted if the number of records is below this value.  It requires a number greater than 1.  The default is 20.       
    'TREE_TERM_MINREC_SPLIT' : '20'
    
-- Specify the maximum number of bins for each attribute.
   For Decision Tree it requires a number between 2 and 2,147,483,647, with the default value of 32. For Random Forest it requires a number between 2 and 254, with the default value of 32.
    'CLAS_MAX_SUP_BINS' : '32'

## Model Evaluation

---

There are many ways to compute model evaluation statistics. One is to use the COMPUTE_LIFT procedure from the DBMS_DATA_MINING library. We'll combine the results from each of the models to display an integrated lift chart. We use the native notebook visualization as well as Python using matplotlib. 

In [33]:
%script

-- Drop apply result and lift tables
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_APPLY_RESULT_RF PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_LIFT_TABLE_RF PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
    
-- Score the data and compute lift
BEGIN
    DBMS_DATA_MINING.APPLY('CLAIMS_CLASS_MODEL_RF','CLAIMS_TEST_DATA','POLICYNUMBER','CLAIMS_APPLY_RESULT_RF');
    DBMS_DATA_MINING.COMPUTE_LIFT('CLAIMS_APPLY_RESULT_RF','CLAIMS_TEST_DATA','POLICYNUMBER','FRAUDFOUND','CLAIMS_LIFT_TABLE_RF','Yes','PREDICTION','PROBABILITY',100);
END;


In [34]:
%script

-- Drop apply result and lift tables
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_APPLY_RESULT_GLM PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_LIFT_TABLE_GLM PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
    
-- Score the data and compute lift
BEGIN
    DBMS_DATA_MINING.APPLY('CLAIMS_CLASS_MODEL_GLM','CLAIMS_TEST_DATA','POLICYNUMBER','CLAIMS_APPLY_RESULT_GLM');
    DBMS_DATA_MINING.COMPUTE_LIFT('CLAIMS_APPLY_RESULT_GLM','CLAIMS_TEST_DATA','POLICYNUMBER','FRAUDFOUND','CLAIMS_LIFT_TABLE_GLM','Yes','PREDICTION','PROBABILITY',100);
END;


In [35]:
%script

-- Drop apply result and lift tables
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_APPLY_RESULT_SVM PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_LIFT_TABLE_SVM PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
    
-- Score the data and compute lift
BEGIN
    DBMS_DATA_MINING.APPLY('CLAIMS_CLASS_MODEL_SVM','CLAIMS_TEST_DATA','POLICYNUMBER','CLAIMS_APPLY_RESULT_SVM');
    DBMS_DATA_MINING.COMPUTE_LIFT('CLAIMS_APPLY_RESULT_SVM','CLAIMS_TEST_DATA','POLICYNUMBER','FRAUDFOUND','CLAIMS_LIFT_TABLE_SVM','Yes','PREDICTION','PROBABILITY',100);
END;


In [36]:
%script

-- Drop apply result and lift tables
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_APPLY_RESULT_DT PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
BEGIN EXECUTE IMMEDIATE 'DROP TABLE CLAIMS_LIFT_TABLE_DT PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/
    
-- Score the data and compute lift
BEGIN
    DBMS_DATA_MINING.APPLY('CLAIMS_CLASS_MODEL_DT','CLAIMS_TEST_DATA','POLICYNUMBER','CLAIMS_APPLY_RESULT_DT');
    DBMS_DATA_MINING.COMPUTE_LIFT('CLAIMS_APPLY_RESULT_DT','CLAIMS_TEST_DATA','POLICYNUMBER','FRAUDFOUND','CLAIMS_LIFT_TABLE_DT','Yes','PREDICTION','PROBABILITY',100);
END;



In [37]:
%sql

SELECT *
FROM CLAIMS_LIFT_TABLE_DT

In [38]:
%sql

CREATE OR REPLACE VIEW ALL_LIFT_DATA AS 
SELECT QUANTILE_NUMBER, ROUND(GAIN_CUMULATIVE*100,2) GAIN_CUMULATIVE, CAST('ALGO_DECISION_TREE' AS VARCHAR(50)) AS ALGO_NAME 
FROM CLAIMS_LIFT_TABLE_DT
UNION
SELECT QUANTILE_NUMBER, ROUND(GAIN_CUMULATIVE*100,2) GAIN_CUMULATIVE, CAST('ALGO_SUPPORT_VECTOR_MACHINES' AS VARCHAR(50)) AS ALGO_NAME 
FROM CLAIMS_LIFT_TABLE_SVM
UNION
SELECT QUANTILE_NUMBER, ROUND(GAIN_CUMULATIVE*100,2) GAIN_CUMULATIVE, CAST('ALGO_RANDOM_FOREST' AS VARCHAR(50)) AS ALGO_NAME 
FROM CLAIMS_LIFT_TABLE_RF
UNION
SELECT QUANTILE_NUMBER, ROUND(GAIN_CUMULATIVE*100,2) GAIN_CUMULATIVE, CAST('ALGO_GENERALIZED_LINEAR_MODEL' AS VARCHAR(50)) AS ALGO_NAME 
FROM CLAIMS_LIFT_TABLE_GLM

In [39]:
%sql

CREATE OR REPLACE VIEW LIFT_STATS AS 
SELECT QUANTILE_NUMBER, GAIN_CUMULATIVE, ALGO_NAME
FROM ALL_LIFT_DATA
UNION 
SELECT QUANTILE_NUMBER, QUANTILE_NUMBER, 'RANDOM_GUESS'
FROM ALL_LIFT_DATA

In [40]:
%sql
SELECT * FROM LIFT_STATS

In [41]:
%sql

SELECT *
FROM CLAIMS_LIFT_TABLE_SVM

In [42]:
%python
import pandas as pd
import oml

import matplotlib
import matplotlib.pyplot as plt


LIFT_DF = oml.sync(view = 'ALL_LIFT_DATA')

ALGOS = LIFT_DF['ALGO_NAME'].drop_duplicates()
lift_df = LIFT_DF.pull()

algo_list = ALGOS.pull()
fig, ax = plt.subplots()


fig.set_figheight(8)
fig.set_figwidth(10)

ax.set_title('Lift Curve')
ax.plot([0, 100], [0, 100], lw=2, linestyle='--', color='red', label='Random guess')
ax.set_xlim([-2, 102])
ax.set_ylim([0, 102])
ax.legend(loc="lower right")

palette = ['blue', 'cyan', 'green', 'orange']

colors = dict()

for i, algo in enumerate(algo_list):
    colors[algo] = palette[i]

for algo in algo_list:

    x = lift_df[lift_df['ALGO_NAME'] == algo]['QUANTILE_NUMBER'].values
    x.sort()
    
    y = lift_df[lift_df['ALGO_NAME'] == algo]['GAIN_CUMULATIVE'].values
    y.sort()
    ax.plot(x, y, color=colors[algo], lw=2, label="%s" % (algo) )
    ax.legend(loc="lower right")
plt.style.use('seaborn')
plt.grid(True)
 
plt.xlabel('Top percentage (%) ', size=13)
plt.ylabel('Recall (%)', size=13)

plt.show()

In [43]:
%python

import pandas as pd
import oml

import matplotlib
import matplotlib.pyplot as plt


LIFT_DF = oml.sync(query = 'SELECT WIDTH_BUCKET(QUANTILE_NUMBER, 1, 10, 9) QUANTILE_BIN, TARGET_DENSITY FROM CLAIMS_LIFT_TABLE_SVM')

LIFT_AGG_DF = LIFT_DF.crosstab(index = 'QUANTILE_BIN', values = 'TARGET_DENSITY', aggfunc=oml.DataFrame.mean)
    
base = LIFT_DF['TARGET_DENSITY'].mean()


decile_df = LIFT_AGG_DF.sort_values('QUANTILE_BIN', ascending = True).pull()

fig, ax = plt.subplots()


fig.set_figheight(8)
fig.set_figwidth(10)


ax.set_title('Waterfall Analysis')
ax.bar(decile_df['QUANTILE_BIN'], decile_df['mean(TARGET_DENSITY)']*100, color='blue', alpha=0.6, label='Model')
ax.axhline(base*100, color='grey', linestyle='--', label='Avg TARGET')
ax.legend(loc="upper right")
    
ax.set_xlabel('Decile', size=13)
ax.set_ylabel('Actual Customers Targeted %', size=13)

plt.style.use('seaborn')
plt.grid(True)
 
plt.show()
    

## Apply Model

---
Applying the model to data, or what's also referred to as scoring, inferencing, or making predictions, can be performed within SQL queries. We first use the PREDICTION_PROBABILITY function with the GLM model to get the probability that fraud was found (target value = 'Yes'). In-database models provide prediction details to explain what predictor values most contributed to the prediction. 

In [45]:
%sql
select round(prob_fraud*100,2) probability_of_fraud,
      rank() over (order by prob_fraud desc) rnk, 
      POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS 
from (select prediction_probability(CLAIMS_CLASS_MODEL_GLM, 'Yes' using *) prob_fraud,POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS
      from CLAIMS )
order by probability_of_fraud desc;      
      


In [46]:
%sql

SELECT * FROM
  (select POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS, round(prob_fraud*100,2) probability_of_fraud,
      rank() over (order by prob_fraud desc) rnk from
      (select POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS, prediction_probability(CLAIMS_CLASS_MODEL_GLM, 'Yes' using *) prob_fraud
          from CLAIMS
))
order by probability_of_fraud desc;

In [47]:
%sql

SELECT * FROM
   (select POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS, round(prob_fraud*100,2) probability_of_fraud,
      rank() over (order by prob_fraud desc) rnk from
      (select POLICYNUMBER, AGEOFPOLICYHOLDER, SEX, MARITALSTATUS, prediction_probability(CLAIMS_CLASS_MODEL_GLM, 'No' using *) prob_fraud
          from CLAIMS
))
order by probability_of_fraud desc;

In [48]:
%sql

SELECT PROBABILITY_OF_FRAUD, RNK, POLICYNUMBER, AGEOFPOLICYHOLDER, SEX 
FROM (
SELECT ROUND(PROB_FRAUD*100,2) PROBABILITY_OF_FRAUD, RANK() OVER (ORDER BY PROB_FRAUD DESC) RNK, POLICYNUMBER, AGEOFPOLICYHOLDER, SEX 
   from
      (SELECT PREDICTION_PROBABILITY(CLAIMS_CLASS_MODEL_GLM, 'Yes' USING *) PROB_FRAUD, POLICYNUMBER, AGEOFPOLICYHOLDER, SEX 
      from CLAIMS)
) a
WHERE RNK <= 5 
ORDER BY PROBABILITY_OF_FRAUD DESC

### Display Prediction Details

---

In [50]:
%script

BEGIN EXECUTE IMMEDIATE 'DROP TABLE SUSPICIOUS_CLAIMS_OAC PURGE';
EXCEPTION WHEN OTHERS THEN NULL; END;
/

In [51]:
%sql

CREATE TABLE SUSPICIOUS_CLAIMS_OAC AS 
SELECT res.* , 
       round(PROB_FRAUD*100,2) probability_of_fraud,
       rank() over (order by prob_fraud desc) rnk 
       from
           (select cl.*, 
            prediction_probability(CLAIMS_CLASS_MODEL_GLM, 'Yes' using *) PROB_FRAUD, 
            prediction_details("CLAIMS_CLASS_MODEL_GLM", 'Yes', 5 ABS USING *) PREDICTIONDETAILS 
            from CLAIMS cl) res
order by probability_of_fraud desc;

In [52]:
%sql

SELECT RNK,PROBABILITY_OF_FRAUD, PREDICTIONDETAILS, POLICYNUMBER, FRAUDFOUND FROM SUSPICIOUS_CLAIMS_OAC;

# End of Script
